In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import os, requests, json, geojson
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, shape
from shapely.ops import unary_union
from shapely.ops import transform
import fiona
import shapely
import pyproj

In [3]:
import boto3

In [4]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data"

In [5]:
WDPA_FILENAME = 'WDPA_Oct2022_Public_shp-polygons.shp'
KBA_FILENAME = 'KBA_data/KBAsGlobal_2022_March_01_POL.shp'

In [6]:
kba_multi = shapely.geometry.MultiPolygon([shapely.geometry.shape(poly['geometry']) for poly in fiona.open(KBA_FILENAME)]) 

In [7]:
wdpa_multi = shapely.geometry.MultiPolygon([shapely.geometry.shape(poly['geometry']) for poly in fiona.open(WDPA_FILENAME)]) 

In [8]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')

# remove cities without tree cover data availability
#tml_not_available_cities = ['BRA-Salvador','MEX-Monterrey']
tml_not_available_cities = []
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(tml_not_available_cities)].reset_index(drop=True)
boundary_georef

,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent
0,ARG-Mendoza,region,ADM3union,ADM3,Mendoza,Argentina,ARG,America
1,ARG-Mar_del_Plata,city,ADM3,ADM4,Mar del Plata city,Argentina,ARG,America
2,ARG-Mar_del_Plata,region,ADM2,NaN,Mar del Plata region,Argentina,ARG,America
3,ARG-Ushuaia,city,ADM4,ADM5,Ushuaia city,Argentina,ARG,America
4,ARG-Ushuaia,region,ADM3,NaN,Ushuaia region,Argentina,ARG,America
5,ARG-Salta,region,ADM2union,ADM3,Salta,Argentina,ARG,America
6,ARG-Buenos_Aires,region,ADM2union,ADM2,Buenos Aires,Argentina,ARG,America
7,BRA-Teresina,city,ADM4union,ADM4,Teresina city,Brazil,BRA,America
8,BRA-Teresina,region,ADM2union,ADM2,Teresina region,Brazil,BRA,America
9,BRA-Florianopolis,city,ADM4union,ADM4,Florianopolis,Brazil,BRA,America


In [9]:
aws_creds = json.loads(open('aws_credentials.json').readline())
ACCESS_KEY = aws_creds['ACCESS_KEY']
SECRET_KEY = aws_creds['SECRET_KEY']

In [10]:
# connect to s3

s3 = boto3.resource(
    service_name = 's3',
    aws_access_key_id = ACCESS_KEY,
    aws_secret_access_key = SECRET_KEY
)

# specify bucket name
bucket_name = 'cities-urbanshift' 

In [60]:
SOURCEDATA = {
    'KBA': kba_multi,
    'WDPA': wdpa_multi
}
for i in range(len(boundary_georef)):
    
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-' + boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-' + boundary_id + '.geojson'
    boundary_geo = requests.get(boundary_path).json()
    temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
    for sourcename in SOURCEDATA:
        sourcegeom = SOURCEDATA[sourcename]
        result = unary_union(MultiPolygon([i for i in sourcegeom.geoms if i.intersects(temp_gdf.iloc[0]['geometry'])]))
        g = gpd.GeoDataFrame(geometry=[result])
        g['boundary_id'] = [boundary_id]
        filepath = "{0}_extracts/{1}-{0}-2022.geojson".format(sourcename, boundary_id)
        g.to_file(filepath, driver='GeoJSON')

        # upload in s3
        s3.meta.client.upload_file(
            filepath, 
            bucket_name, 
            'data/biodiversity/{0}/v_1/{1}-{0}-2022.geojson'.format(sourcename, boundary_id),
            ExtraArgs={'ACL':'public-read'}
        )
            
